# GERIT MATCH

In [1]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

Lade Daten

In [2]:
import pandas as pd

db_hhu = pd.read_csv("data\hhu_db_raw.csv")
db_hhu = db_hhu.sample(n=100, random_state=42)  # random_state für Reproduzierbarkeit


hhu_gerit = pd.read_excel("data/hhu_gerit.xlsx")

Orgas manuell matchen

In [4]:
# Dictionary für schnellen Lookup aus hhu_gerot erstellen
mapping_dict = dict(zip(hhu_gerit["Einrichtung"], hhu_gerit["Einrichtung"]))

# Exakte Matches zuordnen, nicht gefundene bleiben NaN
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].map(mapping_dict)
view(db_hhu)


NameError: name 'view' is not defined

In [ ]:
# def exact_match(query, mapping_dict):
#     if not isinstance(query, str):
#         return None
#     orgs = [org.strip() for org in query.split(';')]  # Aufteilen und trimmen
#     matched_orgs = [mapping_dict.get(org, None) for org in orgs]  # Exakte Übereinstimmung prüfen
#     return "; ".join(filter(None, matched_orgs)) if any(matched_orgs) else None  # Joinen, falls Matches vorhanden

# # Anwenden auf DataFrame
# db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].apply(lambda x: exact_match(x, mapping_dict))


### LLM laden



In [4]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
import openai
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool


# Funktion zur Erstellung von Embeddings mit dem neuen Modell
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Erstelle InMemoryVectorStore für GERIT-Organisationen
vector_store = InMemoryVectorStore(embeddings)

# Alle Organisationen aus GERIT zu den Vektoren hinzufügen
vector_store.add_texts(hhu_gerit["Einrichtung"].tolist())

# Retriever erstellen (Suche nach den 5 ähnlichsten Treffern)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# Beschreibung für den Retriever
description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

# Funktion zum Abgleichen von Organisationen
def match_organisation(query, retriever):
    if not isinstance(query, str):  # Falls query kein String ist (NaN oder float)
        return None  # Oder alternativ "" für leeren String
    
    org_names = query.split(';')  # Aufteilen bei Semikolon (falls mehrere Organisationen)
    best_match = None

    for org in org_names:
        result = retriever.get_relevant_documents(org.strip())  # Korrekte Methode
        if result:
            match = result[0]  # Nimm den besten Treffer
            best_match = match.page_content  # Korrekte Zugriffsmethode

    return best_match




# Zuordnung der Organisationen aus db_hhu zu den GERIT-Organisationen
db_hhu["matched_organisation"] = db_hhu["organisation_mehrere"].apply(lambda x: match_organisation(x, retriever))

# Ergebnisse ausgeben
print(db_hhu)

